In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# === Load Data ===
temporal_df = pd.read_csv("fiber_temporal.csv")
spatial_df = pd.read_csv("fiber_spatial.csv")

# === Label Encoding ===
le = LabelEncoder()
temporal_df['Label_Encoded'] = le.fit_transform(temporal_df['Defect_Label'])
print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

Label mapping: {'Minor Fracture': 0, 'No Defect': 1, 'Severe Fracture': 2, 'Signal Loss': 3}


In [2]:
# === Drop columns ===
segment_ids = temporal_df['Segment_ID'].values
labels = temporal_df['Label_Encoded'].values

# Extract only time-series features (skip Segment_ID, Defect_Label, Label_Encoded)
temporal_features = temporal_df.drop(columns=['Segment_ID', 'Defect_Label', 'Label_Encoded']).values

# === Normalize (Optional) ===
scaler = StandardScaler()
temporal_features_scaled = scaler.fit_transform(temporal_features)

In [3]:
# === Reshape for DL (samples, timesteps, features) ===
# Since we had 4 features per timestep, and 100 timesteps → reshape accordingly
num_timesteps = 100
num_features = 4
temporal_reshaped = temporal_features_scaled.reshape(-1, num_timesteps, num_features)

# === Prepare spatial features ===
# Ensure order matches by merging on Segment_ID
merged_df = pd.merge(temporal_df[['Segment_ID']], spatial_df, on='Segment_ID', how='left')
spatial_features = merged_df.drop(columns=['Segment_ID']).values
spatial_features = StandardScaler().fit_transform(spatial_features)

# === Train-test split ===
X_temp_train, X_temp_test, X_spat_train, X_spat_test, y_train, y_test = train_test_split(
    temporal_reshaped, spatial_features, labels, test_size=0.2, stratify=labels, random_state=42
)

print(f"Temporal train shape: {X_temp_train.shape}")
print(f"Spatial train shape: {X_spat_train.shape}")
print(f"Labels: {np.unique(y_train)}")

Temporal train shape: (400, 100, 4)
Spatial train shape: (400, 3)
Labels: [0 1 2 3]


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Dropout, BatchNormalization, Concatenate, GlobalAveragePooling1D
)
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# === Inputs ===
temporal_input = Input(shape=(100, 4), name="temporal_input")
spatial_input = Input(shape=(3,), name="spatial_input")

# === Temporal Branch ===
x = LSTM(64, return_sequences=True)(temporal_input)
x = GlobalAveragePooling1D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)

# === Spatial Branch ===
y = Dense(32, activation='relu')(spatial_input)
y = Dropout(0.3)(y)
y = BatchNormalization()(y)

# === Fusion ===
combined = Concatenate()([x, y])
z = Dense(64, activation='relu')(combined)
z = Dropout(0.3)(z)
output = Dense(4, activation='softmax')(z)  # 4 defect classes

# === Model ===
model = Model(inputs=[temporal_input, spatial_input], outputs=output)
model.compile(optimizer=Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# === Training ===
history = model.fit(
    [X_temp_train, X_spat_train], y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=32,
    verbose=1
)

# === Evaluation ===
y_pred_probs = model.predict([X_temp_test, X_spat_test])
y_pred = y_pred_probs.argmax(axis=1)

print("=== Classification Report ===")
print(classification_report(y_test, y_pred, target_names=le.classes_))


Epoch 1/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.2123 - loss: 1.6986 - val_accuracy: 0.3625 - val_loss: 1.3465
Epoch 2/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5720 - loss: 0.9800 - val_accuracy: 0.7625 - val_loss: 1.2064
Epoch 3/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6484 - loss: 0.7823 - val_accuracy: 0.7625 - val_loss: 1.0804
Epoch 4/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7027 - loss: 0.6598 - val_accuracy: 0.7500 - val_loss: 0.9834
Epoch 5/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6701 - loss: 0.6463 - val_accuracy: 0.7625 - val_loss: 0.9149
Epoch 6/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7243 - loss: 0.5566 - val_accuracy: 0.7375 - val_loss: 0.8645
Epoch 7/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7337 - loss: 0.5659 - val_accuracy: 0.7625 - val_loss: 0.8336
Epoch 8/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7254 - loss: 0.5526 - val_accuracy: 0.7125 - v